In [1]:
import numpy as np
import pandas as pd
from textacy import text_stats, make_spacy_doc
from datasets import Dataset, load_dataset
import matplotlib.pyplot as plt
import itertools
import torch
import os

In [2]:
df_to_prep = pd.read_csv("Accepted_Rejected_with_scores.csv")

In [ ]:
data_human = pd.DataFrame(load_dataset("argilla/ultrafeedback-binarized-preferences-cleaned")["train"]).sample(n=1000, random_state=42)
data_random = None

## Human Data for fine tuning:

In [4]:
data_human['chosen'] = data_human['chosen'].apply(lambda x: x[1]['content'])
data_human['rejected'] = data_human['rejected'].apply(lambda x: x[1]['content'])
data_human = data_human.loc[:, ['prompt', 'chosen', 'rejected']]

## Metric Data for fine tuning:

In [11]:
data_linear = pd.read_csv('linear_metric_df.csv')
data_linear = data_linear.loc[:, ['prompt', 'chosen', 'rejected', 'accepted_linear_metric', 'rejected_linear_metric', 'diff_linear_metric']]

In [16]:
data_linear

,prompt,chosen,rejected,accepted_linear_metric,rejected_linear_metric,diff_linear_metric
0,Topics: Wound management for general practitio...,General practitioners play a vital role in the...,Wound management is a crucial aspect of genera...,16.343215,-4.498047,20.841262
1,"Part 1. Definition\nIn this task, you are give...",Step 1: Identify the language of the post. The...,Non-hateful\n\nExplanation: This post seems to...,-4.595784,-4.074509,-0.521275
2,You will act as an voice changer. You will cha...,An umbrella has been brought by her on the way.,"Alright, just let me know which form of voice ...",-11.246665,-8.208034,-3.038631
3,Write a well-researched paper on the physiolog...,Title: The Physiological and Psychological Imp...,Title: The Physiological and Psychological Imp...,22.480127,-0.879768,23.359894
4,Create a step-by-step recipe for making fluffy...,"Of course, I'd be happy to help you with that!...",Here is a step-by-step recipe for making fluff...,6.904796,4.521102,2.383693
...,...,...,...,...,...,...
10201,Transform is an emerging brand that delivers m...,"Welcome to Transform, your go-to source for in...","Sure, here's a possible description for the ""A...",-2.827662,-0.951795,-1.875868
10202,"Using Python and the scikit-learn library, how...",Detecting sarcasm in text is a challenging tas...,"Hello! As a helpful and respectful assistant, ...",13.335597,21.171809,-7.836212
10203,"[QUESTION] If ""Two kids stepping stones over t...","Yes, the premise mentions pink plaid pants and...",Confidence: 80%\n\nYou have provided several s...,-8.416876,-7.242787,-1.174089
10204,Write me a letter asking my friend Dan what he...,"Dear Dan,\n\nWe hope this letter finds you wel...",I cannot fulfill your request as it goes again...,3.814516,0.137827,3.676688
